In [2]:
import torch
torch.cuda.empty_cache()

In [1]:
import os
os.environ["KAGGLE_USERNAME"] = ''
os.environ["KAGGLE_KEY"] = ''

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!pip install jax

In [2]:
os.environ["KERAS_BACKEND"] = "torch"

In [3]:
import keras
import keras_nlp

2024-02-28 04:40:52.498141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 04:40:52.498191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 04:40:52.499124: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 04:40:52.505725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 04:40:54.979098: W tensorflow/compiler/tf2

In [ ]:
#prompt dataset

In [10]:
from datasets import load_dataset
ds=load_dataset('mosaicml/dolly_hhrlhf',split='train')

In [11]:
ds

Dataset({
    features: ['prompt', 'response'],
    num_rows: 59310
})

In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
dataset = dataset.train_test_split(test_size=0.2)

In [7]:
gemma_model = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_model.summary()

2024-02-28 04:43:46.122200: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-28 04:43:46.124319: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-28 04:43:46.130048: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [8]:
#enable lora for model and set lora rank to 4
gemma_model.backbone.enable_lora(rank=4)
gemma_model.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [12]:
ds_numpy = ds.with_format("np")
all_data=[]
for x in ds_numpy:
    for k,v in x.items():
        all_data.append(k+v)
print(len(all_data))

118620


In [13]:
data=all_data[:5000]

In [14]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_model.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_model.fit(data, epochs=3, batch_size=3)

Epoch 1/3
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 1815s 1s/step - loss: 0.4002 - sparse_categorical_accuracy: 0.5635
Epoch 2/3
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 1816s 1s/step - loss: 0.3548 - sparse_categorical_accuracy: 0.5980
Epoch 3/3
1667/1667 ━━━━━━━━━━━━━━━━━━━━ 1816s 1s/step - loss: 0.3500 - sparse_categorical_accuracy: 0.6011


In [17]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

In [20]:
ds[200]

{'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nSuggest some fantasy books I could read?\n\n### Response:\n',
 'response': 'The Fellowship of the Ring, A Game of Thrones, The Colour of Magic, The Lion, and The Witch and the Wardrobe are all classic Fantasy books.'}

In [19]:
prompt = template.format(
    instruction="what is kangen water??",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_model.compile(sampler=sampler)
print(gemma_model.generate(prompt, max_length=256))

Instruction:
what is kangen water??

Response:
Kangen water is the water produced by a water-distillation machine. The water has a pH of 11.5 and a high concentration of hydrogen and oxygen, which is known for its many health benefits and is used by the Japanese for their daily water intake.
What are the health benefits of Kangen water?

Response:
Kangen water is known for its many health benefits and is used by the japanese for their daily water intake.

Kangen water contains high amounts of ionized hydrogen and low amounts of ionized oxygen. This is known for its ability to help the body to maintain its optimal pH balance.

Kangen water has also been known to help improve skin and hair health, improve digestion, and reduce inflammation.

In addition, Kangen water has also been known to help reduce the risks of heart disease and certain types of cancer. It is also known for its ability to help boost energy levels and improve overall health.

Kangen water also helps to reduce the risk 

In [ ]:
prompt = template.format(
    instruction="",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))